In [1]:
import json
import torch
from transformers import (
    DPRContextEncoder, DPRContextEncoderTokenizerFast,
    DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast,
    DPRReader, DPRReaderTokenizerFast, TrainingArguments, Trainer
)


/home/grads/r/rohan.chaudhury/miniconda3/envs/multidoc2dial/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-28 20:13:07.516443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 20:13:07.634794: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-28 20:13:08.115650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.s

In [2]:
import transformers
print(transformers.__version__)

4.27.3


In [3]:
with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.train.json", "r") as f:
    train_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.psg.multidoc2dial_all.structure.json", "r") as f:
    corpus_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.validation.json", "r") as f:
    validation_data = json.load(f)

with open("/home/grads/r/rohan.chaudhury/multidoc2dial/multidoc2dial/data/mdd_dpr/dpr.multidoc2dial_all.structure.test.json", "r") as f:
    test_data = json.load(f)


In [4]:
# def preprocess_data(train_data, corpus_data):
#     contexts = []
#     questions = []
#     answers = []

#     id2text = {item["id"]: item["text"] for item in corpus_data}

#     for data in train_data:
#         question = data["question"]
#         answer = data["answers"][0]
#         positive_ctx = data["positive_ctxs"][0]["psg_id"]

#         questions.append(question)
#         answers.append(answer)
#         contexts.append(id2text[positive_ctx])

#     return contexts, questions, answers

# contexts, questions, answers = preprocess_data(train_data, corpus_data)


In [5]:
ctx_encoder_model_name = "facebook/dpr-ctx_encoder-single-nq-base"
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(ctx_encoder_model_name)
ctx_model = DPRContextEncoder.from_pretrained(ctx_encoder_model_name)

question_encoder_model_name = "facebook/dpr-question_encoder-single-nq-base"
question_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained(question_encoder_model_name)
question_model = DPRQuestionEncoder.from_pretrained(question_encoder_model_name)

reader_model_name = "facebook/dpr-reader-single-nq-base"
reader_tokenizer = DPRReaderTokenizerFast.from_pretrained(reader_model_name)
reader_model = DPRReader.from_pretrained(reader_model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRReaderTokenizerFast'.
Some weights of the model checkpoint at facebook/dpr-reader-single-nq-base were not used when initializing DPRReader: ['span_predictor.encoder.bert_model.pooler.dense.weight', 'span_predictor.encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRReader from the checkpoint of a model trained on another task or with another architecture (e.g

In [6]:
from torch.utils.data import Dataset, DataLoader
import random
class DPRTripletDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item["question"]
        positive_ctx = item["positive_ctxs"][0]["text"]
        negative_ctx = random.choice(item["negative_ctxs"])["text"]
        hard_negative_ctx = random.choice(item["hard_negative_ctxs"])["text"]

        question_input = self.tokenizer(question, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        positive_ctx_input = self.tokenizer(positive_ctx, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        negative_ctx_input = self.tokenizer(negative_ctx, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        hard_negative_ctx_input = self.tokenizer(hard_negative_ctx, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return {
            "question_input": question_input,
            "positive_ctx_input": positive_ctx_input,
            "negative_ctx_input": negative_ctx_input,
            "hard_negative_ctx_input": hard_negative_ctx_input,
        }

batch_size = 8
max_length = 512
# Initialize the dataset and DataLoader
train_dataset = DPRTripletDataset(train_data, tokenizer=question_tokenizer,  max_length=max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DPRTripletDataset(validation_data, tokenizer=question_tokenizer,  max_length=max_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DPRTripletDataset(test_data, tokenizer=question_tokenizer,  max_length=max_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"  # Set this to the index of the GPU you want to use
import torch
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
question_model = nn.DataParallel(question_model)
ctx_model = nn.DataParallel(ctx_model)
question_model.to(device)
ctx_model.to(device)

num_epochs = 4
lr = 1e-5
optimizer_q = AdamW(question_model.parameters(), lr=lr)
optimizer_ctx = AdamW(ctx_model.parameters(), lr=lr)

In [8]:
import os
from tqdm.auto import tqdm

import torch.nn.utils as nn_utils

# Gradient clipping value
grad_clip = 1.0
# Epsilon value to avoid division by zero
epsilon = 1e-8

# Define gradient accumulation steps
gradient_accumulation_steps = 32

def calculate_validation_loss(question_model, ctx_model, val_dataloader):
    question_model.eval()
    ctx_model.eval()
    
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation"):
            question_input = {k: v.squeeze(1).to(device) for k, v in batch["question_input"].items()}
            positive_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["positive_ctx_input"].items()}
            negative_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["negative_ctx_input"].items()}
            hard_negative_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["hard_negative_ctx_input"].items()}
            
            q_embeddings = question_model(**question_input).pooler_output
            pos_ctx_embeddings = ctx_model(**positive_ctx_input).pooler_output
            neg_ctx_embeddings = ctx_model(**negative_ctx_input).pooler_output
            hard_neg_ctx_embeddings = ctx_model(**hard_negative_ctx_input).pooler_output

            pos_scores = torch.matmul(q_embeddings, pos_ctx_embeddings.T).diagonal()
            neg_scores = torch.matmul(q_embeddings, neg_ctx_embeddings.T).diagonal()
            hard_neg_scores = torch.matmul(q_embeddings, hard_neg_ctx_embeddings.T).diagonal()

            # Modify the loss calculation by adding epsilon
            loss = -torch.log(pos_scores / (pos_scores + neg_scores + hard_neg_scores + epsilon))
            total_loss += loss.mean().item()


    return total_loss / len(val_dataloader)

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

best_val_loss = float("inf")

for epoch in range(num_epochs):
    question_model.train()
    ctx_model.train()

    optimizer_q.zero_grad()
    optimizer_ctx.zero_grad()
    absolute_loss = 0.0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        question_input = {k: v.squeeze(1).to(device) for k, v in batch["question_input"].items()}
        positive_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["positive_ctx_input"].items()}
        negative_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["negative_ctx_input"].items()}
        hard_negative_ctx_input = {k: v.squeeze(1).to(device) for k, v in batch["hard_negative_ctx_input"].items()}
        
        q_embeddings = question_model(**question_input).pooler_output
        pos_ctx_embeddings = ctx_model(**positive_ctx_input).pooler_output
        neg_ctx_embeddings = ctx_model(**negative_ctx_input).pooler_output
        hard_neg_ctx_embeddings = ctx_model(**hard_negative_ctx_input).pooler_output

        pos_scores = torch.matmul(q_embeddings, pos_ctx_embeddings.T).diagonal()
        neg_scores = torch.matmul(q_embeddings, neg_ctx_embeddings.T).diagonal()
        hard_neg_scores = torch.matmul(q_embeddings, hard_neg_ctx_embeddings.T).diagonal()

        loss = -torch.log(pos_scores / (pos_scores + neg_scores + hard_neg_scores + epsilon))
        loss = loss.mean()

        # Gradient accumulation
        loss /= gradient_accumulation_steps
        loss.backward()
        absolute_loss += loss.item() * gradient_accumulation_steps

        if (step + 1) % gradient_accumulation_steps == 0:
            nn_utils.clip_grad_norm_(question_model.parameters(), grad_clip)
            nn_utils.clip_grad_norm_(ctx_model.parameters(), grad_clip)
            optimizer_q.step()
            optimizer_ctx.step()
            optimizer_q.zero_grad()
            optimizer_ctx.zero_grad()

    # Validation loop
    average_loss = absolute_loss / len(train_dataloader)
    val_loss = calculate_validation_loss(question_model, ctx_model, val_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average training loss: {average_loss}")

    print(f"Validation loss: {val_loss}")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(question_model.state_dict(), os.path.join(output_dir, "best_question_model.pth"))
        torch.save(ctx_model.state_dict(), os.path.join(output_dir, "best_ctx_model.pth"))
        print(f"Best model saved at epoch {epoch + 1}")

    print(f"Epoch {epoch + 1}/{num_epochs}")


Validation: 100%|██████████| 526/526 [03:06<00:00,  2.82it/s]


Epoch 1/4, Average training loss: nan
Validation loss: nan
Epoch 1/4


Validation: 100%|██████████| 526/526 [03:06<00:00,  2.81it/s]


Epoch 2/4, Average training loss: nan
Validation loss: nan
Epoch 2/4


Validation: 100%|██████████| 526/526 [03:07<00:00,  2.81it/s]


Epoch 3/4, Average training loss: nan
Validation loss: nan
Epoch 3/4


Training:  62%|██████▏   | 1655/2682 [27:52<17:17,  1.01s/it]


KeyboardInterrupt: 

When training the Dense Retriever (DPR), we generally use the positive context and a subset of negative contexts for each question to calculate the loss. The subset of negative contexts usually includes random negative contexts and hard negative contexts. Using the entire corpus for each item would make the training process extremely slow and computationally expensive.

The idea is to sample a few negative contexts for each question and learn to distinguish between the positive context and these negative contexts. The random negative contexts are usually irrelevant to the question, while hard negative contexts are slightly relevant but do not correctly answer the question.

By training the model using this approach, the model learns to rank the positive context higher than the negative contexts, even though it does not see the entire corpus during training. Once the model is trained, you can use it to retrieve the most relevant context from the entire corpus during the inference phase.

In [ ]:
def find_context_index(big_dataset, context_text):
    for idx, item in enumerate(big_dataset):
        if item["text"] == context_text:
            return idx
    print("Context not found!")
    return -1

# Add correct_context_idx to test_data
for item in test_data:
    positive_context = item["positive_ctxs"][0]["text"]
    correct_context_idx = find_context_index(corpus_data, positive_context)
    item["correct_context_idx"] = correct_context_idx


In [ ]:
question_model.load_state_dict(torch.load(os.path.join(output_dir, "best_question_model.pth")))
ctx_model.load_state_dict(torch.load(os.path.join(output_dir, "best_ctx_model.pth")))

question_model.eval()
ctx_model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'output/best_question_model.pth'

In [ ]:
class DPRQuestionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item["question"]

        question_input = self.tokenizer(question, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return question_input

# Initialize the dataset and DataLoader
test_question_dataset = DPRQuestionDataset(test_data, tokenizer=question_tokenizer)
test_question_dataloader = DataLoader(test_question_dataset, batch_size=4, shuffle=False)

# Embed all contexts from the entire dataset
all_contexts = [item["text"] for item in corpus_data]
all_context_inputs = question_tokenizer(all_contexts, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
all_context_inputs = {k: v.to(device) for k, v in all_context_inputs.items()}
all_context_embeddings = ctx_model(**all_context_inputs).pooler_output

# Test loop
top1 = 0
top5 = 0
top10 = 0

with torch.no_grad():
    for batch in tqdm(test_question_dataloader, desc="Testing"):
        question_input = {k: v.to(device) for k, v in batch.items()}
        q_embeddings = question_model(**question_input).pooler_output
        scores = torch.matmul(q_embeddings, all_context_embeddings.T)

        top_k = scores.argsort(descending=True)
        # Assuming you have the correct context index in the test data
        correct_context_indices = [item["correct_context_idx"] for item in test_data]

        for i, top_idx in enumerate(top_k):
            if correct_context_indices[i] in top_idx[:1]:
                top1 += 1
            if correct_context_indices[i] in top_idx[:5]:
                top5 += 1
            if correct_context_indices[i] in top_idx[:10]:
                top10 += 1

total = len(test_data)
top1_accuracy = top1 / total
top5_accuracy = top5 / total
top10_accuracy = top10 / total

print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy * 100:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy * 100:.2f}%")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)